In [ ]:
import pandas as pd
import os
import shutil
from pathlib import Path

In [ ]:
swapped_bruk_folder = "/Users/linndfors/study/diploma/data_to_change_T_numbers_NER/bruk_changed"
swapped_ng_folder = "/Users/linndfors/study/diploma/data_to_change_T_numbers_NER/ng_changed"

orig_bruk_folder = "/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/bruk"
orig_ng_folder = "/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/ng"

In [ ]:
def save_indices_to_meta(source_path, indices):
    meta_filename = source_path.replace(".ann", ".meta")
    with open(meta_filename, 'w', encoding='utf-8') as meta_file:
        for index in indices.values():
            meta_file.write(f"{index}\n")

In [ ]:
def find_entity_line(text, start_idx, end_idx):
    lines = text.split("\n")
    char_index = 0

    for i, line in enumerate(lines):
        line_start = char_index
        line_end = line_start + len(line)
        
        if line_start <= start_idx < line_end:
            return i, line
        
        char_index = line_end + 1
    print("no match for the", start_idx, end_idx)
    return None, None 

def find_orig_sent_id(orig_path_ann, tag):
    with open(orig_path_ann, 'r', encoding='utf-8') as file:
            ann_lines = file.readlines()

    orig_path_txt = orig_path_ann.replace(".ann", ".txt")
    with open(orig_path_txt, 'r', encoding='utf-8') as file:
            text = file.read()

    for line in ann_lines:
        parts = line.strip().split('\t')
        if len(parts) >= 3:
            if parts[0] == tag:
                start_ind, end_ind = int(parts[2]), int(parts[3])
                sent_ind = find_entity_line(text, start_ind, end_ind)
                return sent_ind[0]
            
def find_indeces(source_path, orig_path):
    filename = os.path.basename(source_path) 

    orig_filename = filename.replace("_1_filtered.ann", ".ann")
    orig_file_path = os.path.join(orig_path, orig_filename)
    tags_with_indx = {}

    with open(source_path, 'r', encoding='utf-8') as file:
            ann_lines = file.readlines()

    for line in ann_lines:
        parts = line.strip().split('\t')
        if len(parts) >= 3:
            if parts[1] in ["JOB", "PERS"]:
                tag = parts[0]
                orig_sent_id = find_orig_sent_id(orig_file_path, tag)
                if orig_sent_id is None:
                    print(f"Cannot find a relevant index for file {source_path}, the row: {parts}")
                    return
                else:
                    tags_with_indx[tag] = orig_sent_id

    save_indices_to_meta(source_path, tags_with_indx)


In [ ]:
# find_indeces("/Users/linndfors/study/diploma/data_to_change_T_numbers_NER/bruk_changed/0ac5140eb732_1_filtered.ann", "/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/bruk")

In [ ]:
def process_ann_files(directory, orig_path):
    for filename in os.listdir(directory):
        if filename.endswith(".ann"):
            source_path = os.path.join(directory, filename)
            find_indeces(source_path, orig_path)

In [ ]:
def copy_meta_files(src_dir, dest_dir):
    Path(dest_dir).mkdir(parents=True, exist_ok=True)
    
    for filepath in Path(src_dir).rglob('*.meta'):
        if filepath.is_file():
            modified_filepath = str(filepath).replace("_1_filtered", "_1")
            
            dest_filepath = Path(dest_dir) / Path(modified_filepath).name
            shutil.copy(filepath, dest_filepath)
            # print(f"Copied: {dest_filepath.name} to {dest_filepath}")

### bruk

In [ ]:
process_ann_files("/Users/linndfors/study/diploma/data_to_change_T_numbers_NER/bruk_changed", "/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/bruk")

In [ ]:
src_directory = "/Users/linndfors/study/diploma/data_to_change_T_numbers_NER/bruk_changed"
dest_directory = "/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/bruk_changed"

copy_meta_files(src_directory, dest_directory)

### ng

In [ ]:
process_ann_files("/Users/linndfors/study/diploma/data_to_change_T_numbers_NER/ng_changed", "/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/ng")

In [ ]:
src_directory = "/Users/linndfors/study/diploma/data_to_change_T_numbers_NER/ng_changed"
dest_directory = "/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/ng_changed"

copy_meta_files(src_directory, dest_directory)

# To v2.0-swapped

In [ ]:
def rename_and_move_files(src_dir, dest_dir):
    Path(dest_dir).mkdir(parents=True, exist_ok=True)

    for filepath in Path(src_dir).rglob('*_1*'):
        new_name = filepath.name.replace('_1', '-swapped')
        new_path = Path(dest_dir) / new_name
        
        shutil.copy(filepath, new_path)
        # print(f'Copied & Renamed: {filepath.name} -> {new_name}')

In [ ]:
src_directory = "/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/bruk_changed"
dest_directory = "/Users/linndfors/study/diploma/ner_for_fem/data/v2.0-swapped/bruk"

rename_and_move_files(src_directory, dest_directory)

In [ ]:
src_directory = "/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/ng_changed"
dest_directory = "/Users/linndfors/study/diploma/ner_for_fem/data/v2.0-swapped/ng"

rename_and_move_files(src_directory, dest_directory)

add original files

In [ ]:
def copy_all_files(src_dir, dest_dir):
    Path(dest_dir).mkdir(parents=True, exist_ok=True)

    for file in Path(src_dir).iterdir():
        if file.is_file():
            shutil.copy(file, dest_dir)
            # print(f"Copied: {file.name} -> {dest_dir}")

In [ ]:
src_directory = "/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/ng"
dest_directory = "/Users/linndfors/study/diploma/ner_for_fem/data/v2.0-swapped/ng"

copy_all_files(src_directory, dest_directory)

In [ ]:
src_directory = "/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/bruk"
dest_directory = "/Users/linndfors/study/diploma/ner_for_fem/data/v2.0-swapped/bruk"

copy_all_files(src_directory, dest_directory)